In [6]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('wordnet')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
load_dotenv('checkpoint.env')

True

In [16]:
from newsapi import NewsApiClient

newsapi_key = os.getenv('NEWSAPI_KEY')
newsapi = NewsApiClient(api_key=newsapi_key)

# TEST:
# type(newsapi_key)

In [32]:
def getHeadline(ticker):
    return newsapi.get_everything(
    q=ticker,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# TEST:
# aapl_headlines = getHeadline('aapl')

In [41]:
def getSentimentTable(headlines):
    sentiments = []
    
    for article in headlines['articles']:
        try:
            title = article["title"]
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]

            sentiments.append({
                "title": title,
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu

            })
        except AttributeError:
            pass
    
    sentiment_df = pd.DataFrame(sentiments)
    cols = ["date", "title", "text", "compound", "positive", "negative", "neutral"]
    sentiment_df = sentiment_df[cols]
    return sentiment_df.sort_values(by="date", ascending=False)

# TEST:
# aapl_sentiment_df = getSentimentTable(aapl_headlines)
# aapl_sentiment_df.head()

,date,title,text,compound,positive,negative,neutral
99,2021-07-01,Kelley Blue Book: The 2021 Porsche Cayenne is ...,The Cayenne has been on the market for only tw...,0.7227,0.129,0.000,0.871
54,2021-07-01,Capitol Report: Amazon’s fight to sideline its...,Amazon filed a motion Wednesday with the Feder...,0.1779,0.055,0.000,0.945
64,2021-07-01,: FTC has a chance for a do-over in its ‘fiasc...,Could the second time be the charm for the Fed...,0.0584,0.069,0.063,0.868
12,2021-07-01,: Ex-Trump adviser Jason Miller to launch GETT...,"Jason Miller, former senior adviser to Preside...",0.2960,0.068,0.000,0.932
2,2021-07-01,4 Consumer Electronics Stocks to Buy for the S...,This story originally appeared on StockNews\r\...,0.7184,0.214,0.043,0.743
